In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

# = = = = = = = = = = = = = = =

from IPython.display import clear_output
from datetime import datetime

In [2]:
param_date = '20240715'

In [3]:
import pandas as pd

output_correct, output_error = pd.DataFrame(), pd.DataFrame()

# = = = = = = = = = = = = = = =

input_ = pd.read_excel(f'./{param_date}/2.year.xlsx',
                       header=0,
                       dtype=str).fillna('')

input_['Make No'] = input_['Make No'].astype(int)
input_['Year'] = input_['Year'].astype(int)

print(f'总数量：{len(input_)}')
print()

# = = = = = = = = = = = = = = =

work = Queue()
for a in range(len(input_)):
    work.put_nowait(a)

# = = = = = = = = = = = = = = =

def crawler():
    global output_correct, output_error

    while not work.empty():
        a = work.get_nowait()

        # = = = = = = = = = = = = = = =

        crawler_status = 'error'

        # = = = = = = = = = = = = = = =

        import requests
        import json

        import sys
        sys.path.append('../../00.Tools')
        from crawler_configuration import get_header, get_proxy

        payload = {'jsn': {'tab': 'catalog',
                           'make': input_.loc[a, 'Make'],
                           'year': int(input_.loc[a, 'Year']),
                           'nodetype': 'year'}}

        data = {'func': 'navnode_fetch',
                'payload': json.dumps(payload),
                'api_json_request': '1'}

        try:
            b = 0
            while True:

                while True:
                    b += 1
                    try:
                        resp = requests.post('https://www.rockauto.com/catalog/catalogapi.php',
                                             data=data,
                                             headers=get_header(),
                                             proxies=get_proxy(),
                                             timeout=(10, 10))
                        break
                    except:
                        continue

                if resp.status_code == 200:
                    json_ = resp.json()
                    if 'redirect_to_url' in json_:
                        continue
                    elif json_['jsn']['tab'] == 'catalog' and json_['jsn']['make'] == input_.loc[a, 'Make'] and json_['jsn']['year'] == input_.loc[a, 'Year'] and json_['jsn']['nodetype'] == 'year' and 'collected_javascript' in json_:
                        break

            # = = = = = = = = = = = = = = =

            from bs4 import BeautifulSoup
            from lxml import etree

            soup = BeautifulSoup(json_['html_fill_sections']['navchildren[]'], 'lxml')
            html = etree.HTML(str(soup))

            # = = = = = = = = = = = = = = =

            list_json = html.xpath('//input[contains(@id, "jsn[__GIP__")]/@value')

            # = = = = = = = = = = = = = = =

            list_model = [json.loads(json_)['model'] for json_ in list_json]

            # = = = = = = = = = = = = = = =

            if not list_model:
                raise

            # = = = = = = = = = = = = = = =

            df_temp = pd.DataFrame({'Make No': input_.loc[a, 'Make No'],
                                    'Make': input_.loc[a, 'Make'],
                                    'Year': input_.loc[a, 'Year'],
                                    'Model No': [i+1 for i in range(len(list_model))],
                                    'Model': list_model})

            output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

            # = = = = = = = = = = = = = = =

            crawler_status = 'ok'
        except:
            df_temp = pd.DataFrame([{'Make No': input_.loc[a, 'Make No'],
                                     'Make': input_.loc[a, 'Make'],
                                     'Year': input_.loc[a, 'Year']}])

            output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')

        # = = = = = = = = = = = = = = =

        if work.qsize() % 1_000 == 0 and work.qsize() != 0:
            clear_output()

        print(f'''[状态：{crawler_status}，尝试次数：{b}] - {input_.loc[a, 'Make No']}.{input_.loc[a, 'Make']} > {input_.loc[a, 'Year']}\n[剩余数量：{work.qsize()}] - [当前时间：{datetime.now().strftime('%H:%M:%S')}]\n''')

# = = = = = = = = = = = = = = =

list_task = []
for _ in range(20):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print('输出ing...')
print()
if not output_correct.empty:
    output_correct = output_correct.sort_values(by=['Make No', 'Year', 'Model No'],
                                                ascending=[True, False, True],
                                                ignore_index=True)
    output_correct.to_excel(f'''./{param_date}/3.model-{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx''', index=False)
if not output_error.empty:
    output_error = output_error.sort_values(by=['Make No', 'Year'],
                                            ascending=[True, False],
                                            ignore_index=True)
    output_error.to_excel(f'./{param_date}/model_error.xlsx', index=False)
    print('爬虫存在error')
    print()
print('Done ~')

[状态：ok，尝试次数：  1] - 216.SAAB > 2008
[剩余数量：1000] - [当前时间：10:22:55]

[状态：ok，尝试次数：  1] - 215.ROVER > 1953
[剩余数量：999] - [当前时间：10:22:55]

[状态：ok，尝试次数：  1] - 216.SAAB > 1996
[剩余数量：998] - [当前时间：10:22:56]

[状态：ok，尝试次数：  1] - 216.SAAB > 1994
[剩余数量：997] - [当前时间：10:22:56]

[状态：ok，尝试次数：  1] - 216.SAAB > 2001
[剩余数量：996] - [当前时间：10:22:56]

[状态：ok，尝试次数：  1] - 216.SAAB > 1995
[剩余数量：995] - [当前时间：10:22:56]

[状态：ok，尝试次数：  1] - 216.SAAB > 2002
[剩余数量：994] - [当前时间：10:22:56]

[状态：ok，尝试次数：  1] - 216.SAAB > 1990
[剩余数量：993] - [当前时间：10:22:56]

[状态：ok，尝试次数：  1] - 215.ROVER > 1954
[剩余数量：992] - [当前时间：10:22:56]

[状态：ok，尝试次数：  1] - 216.SAAB > 1993
[剩余数量：991] - [当前时间：10:22:57]

[状态：ok，尝试次数：  1] - 216.SAAB > 1997
[剩余数量：990] - [当前时间：10:22:57]

[状态：ok，尝试次数：  1] - 216.SAAB > 1986
[剩余数量：989] - [当前时间：10:22:57]

[状态：ok，尝试次数：  1] - 215.ROVER > 1974
[剩余数量：988] - [当前时间：10:22:57]

[状态：ok，尝试次数：  1] - 216.SAAB > 1991
[剩余数量：987] - [当前时间：10:22:57]

[状态：ok，尝试次数：  1] - 216.SAAB > 1981
[剩余数量：986] - [当前时间：10:22:57]

[状态：ok，尝试次数：  1] - 21